# Estudio de índices climáticos
## Pablo Lavín

In [1]:
# Cargamos paquetes

library(repr)
library(dplyr)

library(abind)
library(loadeR)
library(transformeR)
library(convertR)
library(visualizeR)
library(downscaleR)
library(climate4R.UDG)
library(climate4R.climdex)
library(climate4R.indices)
library(easyVerification)

library(lattice)
library(magrittr)
library(grid)
library(gridExtra)
library(RColorBrewer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0

In [2]:
# Region de estudio

lon = c(-10, 5)
lat = c(35,44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

## Cargo datos ERA5-Land

In [3]:
for (m in 1:12) {
    # Temperatura mínima
    assign(paste0("era5_tmin_", sprintf("%02d", m)),
           readRDS(paste0("Data_ERA5-Land/era5_tmin_", sprintf("%02d", m), ".rds")))

    # Precipitación
    assign(paste0("era5_pr_", sprintf("%02d", m)),
           readRDS(paste0("Data_ERA5-Land/era5_pr_", sprintf("%02d", m), ".rds")))
}

## Cargo datos PTI

In [4]:
for (m in 1:12) {
    # Temperatura mínima
    assign(paste0("pti_tmin_", sprintf("%02d", m)),
           readRDS(paste0("Data_PTI-v0/pti_tmin_", sprintf("%02d", m), ".rds")))

    # Precipitación
    assign(paste0("pti_pr_", sprintf("%02d", m)),
           readRDS(paste0("Data_PTI-v0/pti_pr_", sprintf("%02d", m), ".rds")))
}

## Máscara para los datos

In [5]:
## Calculo el número de días que tmax > 22 grados (solo para la estructura del grid)
nd_obs = indexGrid(tx = era5_hurs_01, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(era5_tmean_01),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

ERROR: Error: objeto 'era5_hurs_01' no encontrado


## Funciones auxiliares

In [6]:
# Calcula la media anual de días que cumplen una condición conjunta de temperatura
# y humedad relativa a partir de datos diarios. Opcionalmente aplica una máscara
# geográfica (por ejemplo, para tierra).
#
# Esta función se utilizado para datos de observaciones con dimeniones [time, lat, lon]
#
# @param tas_obs Lista con los datos diarios de temperatura y su información asociada.
# @param pr_obs Lista con los datos diarios de precipitación y su información asociada.
# @param temp_thresh Umbral de temperatura (por defecto 10 ºC).
# @param pr_thresh Umbral de precipitación (por defecto 10 mm).
#
# @return Lista tipo "grid" con la media anual de días que cumplen la condición.
#

compute_masked_obs = function(tas_obs, pr_obs, temp_thresh = 10, pr_thresh = 10) {
    
    # Vector de fechas diarias
    dates = as.Date(tas_obs$Dates$start)
    years = factor(format(dates, "%Y"))
    unique_years = levels(years)
    n_years = length(unique_years)
    nlat = dim(tas_obs$Data)[2]
    nlon = dim(tas_obs$Data)[3]
    
    # Condición conjunta: tas > temp_thresh y pr > pr_thresh
    mask = (tas_obs$Data > temp_thresh) & (pr_obs$Data >= pr_thresh)
    
    # Array [time(year), lat, lon]
    annual_days = array(NA, dim = c(n_years, nlat, nlon))
    
    for (y in 1:n_years) {
        idx = which(years == unique_years[y])
        annual_days[y,,] = apply(mask[idx,,], c(2,3), sum, na.rm = TRUE)
    }
    
    # Reconstruyo el grid
    grid = list()
    grid$Data = annual_days
    attr(grid$Data, "dimensions") = c("time", "lat", "lon")
    grid$xyCoords = tas_obs$xyCoords
    grid$Variable = tas_obs$Variable
    grid$Dates = tas_obs$Dates
    class(grid) = "grid"
    
    # Aplico la máscara de los datos
    grid = gridArithmetics(grid, mask.bin.spain, operator = "*")
    
    return(grid)
}

## Riesgo aparición mildiu (Nº días con Tmin > 10ºC y PR > 10)

### ERA5-Land

In [12]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("era5_tmin_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("tmin_era5_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(-10, 25, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("tmin_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("tmin_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Temperatura mínima media (1981-2021) (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

In [24]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("era5_pr_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("pr_era5_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 15, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("pr_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("pr_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Precipitación media (1981-2021) (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

In [7]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    tmin_var = get(paste0("era5_tmin_", mes))
    pr_var   = get(paste0("era5_pr_", mes))
  
    # Aplico la máscara y la guardo con el sufijo
    assign(paste0("grid_masked_", mes),
           compute_masked_obs(tmin_var, pr_var,
                              temp_thresh = 10,
                              pr_thresh   = 10))
  
    # Recupero el objeto recién creado
    grid_masked_obs = get(paste0("grid_masked_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("nd_obs_era5_", mes), spatialPlot(
        climatology(grid_masked_obs), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 31, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [8]:
png("ndays_riesgo_mildiu_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("nd_obs_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Número de días de riesgo de mildiu (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

### PTI-grid-v0

In [19]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("pti_tmin_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("tmin_pti_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(-10, 25, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("tmin_pti_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("tmin_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Temperatura mínima media (1981-2021) (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

In [25]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("pti_pr_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("pr_pti_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 15, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("pr_pti_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("pr_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Precipitación media (1981-2021) (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

In [9]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    tmin_var = get(paste0("pti_tmin_", mes))
    pr_var   = get(paste0("pti_pr_", mes))
  
    # Aplico la máscara y la guardo con el sufijo
    assign(paste0("grid_masked_", mes),
           compute_masked_obs(tmin_var, pr_var,
                              temp_thresh = 10,
                              pr_thresh   = 10))
  
    # Recupero el objeto recién creado
    grid_masked_obs = get(paste0("grid_masked_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("nd_obs_pti_", mes), spatialPlot(
        climatology(grid_masked_obs), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 31, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [10]:
png("ndays_riesgo_mildiu_ptiv0_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("nd_obs_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Número de días de riesgo de mildiu (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2